# EDA on taxi_trip_pricing

In [4]:
import pandas as pd

data = pd.read_csv("../data/raw/taxi_trip_pricing.csv")
data.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


In [6]:
data.shape

(1000, 11)

In [7]:
data_notna = pd.read_csv("../data/raw/taxi_trip_pricing.csv").dropna()
data_notna.shape

(562, 11)

### Nearly 5% of every column is NaN